In [1]:
import pickle, tqdm, time, os, tensorflow_hub as hub, numpy as np, pandas as pd
from pymongo import MongoClient
from pprint import pprint

In [2]:
def getMongoUri(auth_filename='auths/mongo_uri.pkl'):
    with open(auth_filename, 'rb') as file:
        pwd = pickle.load(file)
    return pwd['mongo_uri']

def createMongoClient(db='aita'):
    mongo_uri = getMongoUri()
    client = MongoClient(mongo_uri)
    db = client.aita
    return client, db

In [3]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed(['testing'])
print('model cached')

model cached


In [4]:
client, db = createMongoClient()
postsCollection = db.clean.posts
NUM_DOCS = postsCollection.count_documents({})

In [ ]:
allPosts = []
for post in tqdm.tqdm_notebook(postsCollection.find(projection={'_id': 0}), total=NUM_DOCS):
    allPosts.append(post)

/Users/alamjeetsingh/opt/anaconda3/envs/cs448/lib/python3.6/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/490330 [00:00<?, ?it/s]

In [ ]:
# calculates end index for a particular iteration for looping through documents in batches
def calcEndIdx(start_idx, batch_size, ndocs):
    end_idx = start_idx + batch_size
    end_idx = ndocs if end_idx > ndocs - 1 else end_idx
    return end_idx

In [ ]:
selftexts = [x['selftext'] for x in allPosts]

In [ ]:
# convert text to embeddings in batches (model can handle multiple texts at once)
# batch size depends on compute power
embeddings = [] # empty array to store embeddings as we iterate through docs
BATCH_SIZE = 1000

for start_idx in tqdm.tqdm_notebook(range(0, NUM_DOCS, BATCH_SIZE)):
    end_idx = calcEndIdx(start_idx, BATCH_SIZE, NUM_DOCS)
    curr_embeddings = embed(selftexts[start_idx:end_idx]).numpy()
    embeddings.append(curr_embeddings)
    
embeddings = np.concatenate(embeddings) # convert batched arrays to shape (N, Vector Size)

In [ ]:
len(embeddings)

In [ ]:
embeddings.shape

In [ ]:
for i in range(len(allPosts)):
    allPosts[i]['selftextVector'] = embeddings[i].tolist()

In [ ]:
cleanPostsV2 = db.clean.posts.v2
cleanPostsV2.insert_many(allPosts)